## Importação de Bibliotecas

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Image
import cv2
import requests
from bs4 import BeautifulSoup
import io
import numpy as np
from PIL import Image
import face_recognition
import datetime

In [2]:
display(HTML("<div style='text-align:center'><img src='https://apiesltv.imgix.net/images/team/logo/180_6389fd40-d1b3-4bd3-9a64-6ede7e24bd38.png?auto=compress&w=400' width='150px'/></div>"))
display(HTML("<h1 style='text-align:center; color:#ff3d00;'>FURIA FAN INSIGHTS</h1>"))
display(HTML("<p style='text-align:center; font-size:16px;'>Sistema de verificação e engajamento de fãs</p><hr>"))

## Funções Auxiliares

### Função para validar documentos com detecção de rosto

In [3]:
def validate_document(file_content):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = Image.open(io.BytesIO(file_content))
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return len(faces) > 0

### Função para validar links de perfis

In [4]:
def validate_profile_link(link, interests):
    try:
        response = requests.get(link, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text().lower()
        return any(interest.lower() in text for interest in interests.split(','))
    except:
        return False

### Função para comparar a foto do documento com a selfie da pessoa

In [5]:
def compare_selfie_with_document(selfie_content, document_content, threshold=0.6):
    try:
        selfie_img = face_recognition.load_image_file(io.BytesIO(selfie_content))
        document_img = face_recognition.load_image_file(io.BytesIO(document_content))

        selfie_face_locations = face_recognition.face_locations(selfie_img)
        document_face_locations = face_recognition.face_locations(document_img)

        if not selfie_face_locations or not document_face_locations:
            return False, 1.0  # Não encontrou faces em uma ou ambas as imagens

        # Extrai os encodings (características) das faces
        selfie_face_encoding = face_recognition.face_encodings(selfie_img, selfie_face_locations)[0]
        document_face_encoding = face_recognition.face_encodings(document_img, document_face_locations)[0]

        # Calcula a distância entre as faces (menor valor = mais similar)
        face_distance = face_recognition.face_distance([selfie_face_encoding], document_face_encoding)[0]

        # Compara se a distância é menor que o limiar
        match = face_distance <= threshold

        return match, face_distance
    except Exception as e:
        print(f"Erro ao comparar faces: {e}")
        return False, 1.0

## Seções do formulário: 

### Seção 1: Dados Pessoais

In [6]:
def validate_cpf(cpf):
    cpf = ''.join(filter(str.isdigit, cpf))
    
    if len(cpf) != 11:
        return False
    
    if cpf == cpf[0] * 11:
        return False
    
    soma = 0
    for i in range(9):
        soma += int(cpf[i]) * (10 - i)
    resto = soma % 11
    digito1 = 0 if resto < 2 else 11 - resto
    
    if digito1 != int(cpf[9]):
        return False

    soma = 0
    for i in range(10):
        soma += int(cpf[i]) * (11 - i)
    resto = soma % 11
    digito2 = 0 if resto < 2 else 11 - resto
    
    return digito2 == int(cpf[10])

name_input = widgets.Text(description="Nome:")
address_input = widgets.Text(description="Endereço:")
cpf_input = widgets.Text(description="CPF:")
cpf_output = widgets.Output()  # Output para mostrar validação do CPF

display(name_input, address_input, cpf_input, cpf_output)

def on_cpf_change(change):
    with cpf_output:
        clear_output()
        if cpf_input.value:
            cpf = cpf_input.value
            if validate_cpf(cpf):
                print("✅ CPF válido")
            else:
                print("❌ CPF inválido. Por favor, insira um CPF válido.")

cpf_input.observe(on_cpf_change, names='value')

Text(value='', description='Nome:')

Text(value='', description='Endereço:')

Text(value='', description='CPF:')

Output()

### Seção 2: Interesses

In [7]:
interests_input = widgets.Textarea(description="Interesses (ex: jogos, times):", rows=3)
display(interests_input)

Textarea(value='', description='Interesses (ex: jogos, times):', rows=3)

### Seção 3: Upload de Documento

In [8]:
### Seção 3: Upload de Documento

doc_upload = widgets.FileUpload(
    description="Upload de Documento", 
    accept=".jpg,.jpeg,.png,.pdf,.bmp",  # Aceita mais formatos
    multiple=False  # Permite apenas um arquivo
)
doc_output = widgets.Output()
display(doc_upload, doc_output)

# Variável para armazenar o conteúdo do documento
document_content = None

def on_doc_upload_change(change):
    global document_content
    with doc_output:
        clear_output()
        if doc_upload.value:
            try:
                # Tenta extrair o conteúdo do arquivo
                if isinstance(doc_upload.value, tuple) and len(doc_upload.value) > 0:
                    # Se for uma tupla, vamos tentar acessar o conteúdo diretamente do objeto
                    if hasattr(doc_upload.value[0], 'content'):
                        file_content = doc_upload.value[0].content  # Acessa o atributo content
                    elif hasattr(doc_upload.value[0], '_data'):
                        file_content = doc_upload.value[0]._data  # Tenta acessar _data
                    elif hasattr(doc_upload.value[0], 'data'):
                        file_content = doc_upload.value[0].data  # Tenta acessar data
                    elif hasattr(doc_upload.value[0], 'value'):
                        file_content = doc_upload.value[0].value  # Tenta acessar value
                    else:
                        # Último recurso: tentativa de conversão direta
                        print("Tentando conversão direta do objeto para bytes...")
                        file_content = bytes(doc_upload.value[0])
                else:
                    print("Formato de upload não reconhecido.")
                    return
                
                document_content = file_content
                
                # Mostra informações sobre o arquivo
                print(f"Documento carregado! Tamanho: {len(file_content) / 1024:.1f} KB")
                
                if validate_document(file_content):
                    print("Validando documento com IA... Face detectada com sucesso!")
                else:
                    print("Validando documento com IA... Aviso: Nenhum rosto foi detectado.")
                    print("Você ainda pode prosseguir, mas a verificação pode não funcionar corretamente.")
            except Exception as e:
                print(f"Erro ao processar arquivo: {e}")
                print(f"Tipo do objeto: {type(doc_upload.value)}")
                if isinstance(doc_upload.value, tuple) and len(doc_upload.value) > 0:
                    print(f"Tipo do primeiro elemento: {type(doc_upload.value[0])}")
                    print(f"Atributos disponíveis: {dir(doc_upload.value[0])}")
                document_content = None

doc_upload.observe(on_doc_upload_change, names='value')

FileUpload(value=(), accept='.jpg,.jpeg,.png,.pdf,.bmp', description='Upload de Documento')

Output()

### Seção 4: Redes Sociais

In [14]:
# Seção 4: Redes Sociais - Versão melhorada
social_input = widgets.Text(
    description="Redes Sociais:",
    placeholder="@usuario1, @usuario2, ..."
)
social_help = widgets.HTML(
    value="<small style='color:#666'>Separe múltiplos perfis com vírgulas. Ex: @furia_cs, @usuario_twitter</small>"
)
social_output = widgets.Output()

display(social_input, social_help, social_output)

def on_social_change(change):
    with social_output:
        clear_output()
        if social_input.value:
            # Conta quantos perfis foram informados
            perfis = [p.strip() for p in social_input.value.split(',') if p.strip()]
            num_perfis = len(perfis)
            
            if num_perfis > 0:
                print(f"Simulando leitura de interações... Encontradas {num_perfis * 10} interações relacionadas a esports.")
                
                # Lista cada perfil detectado
                print("\nPerfis detectados:")
                for i, perfil in enumerate(perfis, 1):
                    print(f"{i}. {perfil}")
            
social_input.observe(on_social_change, names='value')

Text(value='', description='Redes Sociais:', placeholder='@usuario1, @usuario2, ...')

HTML(value="<small style='color:#666'>Separe múltiplos perfis com vírgulas. Ex: @furia_cs, @usuario_twitter</s…

Output()

### Seção 5: Links de Perfis

In [18]:
profile_link_input = widgets.Text(
    description="Link de Perfil:",
    placeholder="https://site1.com, https://site2.com, ..."
)
profile_help = widgets.HTML(
    value="<small style='color:#666'>Separe múltiplos links com vírgulas. Ex: https://steamcommunity.com/id/seuuser, https://faceit.com/pt-br/players/seuuser</small>"
)
profile_output = widgets.Output()

display(profile_link_input, profile_help, profile_output)

def on_link_change(change):
    with profile_output:
        clear_output()
        if profile_link_input.value:
            links = [link.strip() for link in profile_link_input.value.split(',') if link.strip()]
            
            if not links:
                return
                
            # Valida cada link
            valid_links = []
            invalid_links = []
            
            print("Validando links de perfil...")
            
            for link in links:
                if validate_profile_link(link, interests_input.value):
                    valid_links.append(link)
                else:
                    invalid_links.append(link)
            
            # Exibe os resultados
            if valid_links:
                print("\n✅ Links com conteúdo relevante:")
                for i, link in enumerate(valid_links, 1):
                    print(f"{i}. {link}")
            
            if invalid_links:
                print("\n❌ Links sem conteúdo relevante ou inválidos:")
                for i, link in enumerate(invalid_links, 1):
                    print(f"{i}. {link}")
                    
            # Resumo da validação
            total = len(links)
            valid = len(valid_links)
            print(f"\nResumo: {valid}/{total} links validados com sucesso.")
            
            if valid > 0:
                print(f"Análise de IA: {valid} perfis correspondem aos seus interesses em esports.")

profile_link_input.observe(on_link_change, names='value')

Text(value='', description='Link de Perfil:', placeholder='https://site1.com, https://site2.com, ...')

HTML(value="<small style='color:#666'>Separe múltiplos links com vírgulas. Ex: https://steamcommunity.com/id/s…

Output()

### Seção 6: Verificação de Identidade por Selfie

In [11]:
### Seção 6: Verificação de Identidade por Selfie

selfie_upload = widgets.FileUpload(
    description="Upload de Selfie", 
    accept=".jpg,.jpeg,.png,.bmp",  # Formatos de imagem
    multiple=False  # Permite apenas um arquivo
)
verify_button = widgets.Button(description="Verificar Identidade")
selfie_output = widgets.Output()  # Adicionado para mostrar feedback do upload
verify_output = widgets.Output()

display(selfie_upload, selfie_output, verify_button, verify_output)  # Adicionei selfie_output ao display

# Variável para armazenar o conteúdo da selfie
selfie_content = None

# Manipulador para o upload de selfie
def on_selfie_upload_change(change):
    global selfie_content
    with selfie_output:  # Use a saída específica para o upload
        clear_output()
        if selfie_upload.value:
            try:
                # Tenta extrair o conteúdo do arquivo
                if isinstance(selfie_upload.value, tuple) and len(selfie_upload.value) > 0:
                    # Se for uma tupla, vamos tentar acessar o conteúdo diretamente do objeto
                    if hasattr(selfie_upload.value[0], 'content'):
                        file_content = selfie_upload.value[0].content  # Acessa o atributo content
                    elif hasattr(selfie_upload.value[0], '_data'):
                        file_content = selfie_upload.value[0]._data  # Tenta acessar _data
                    elif hasattr(selfie_upload.value[0], 'data'):
                        file_content = selfie_upload.value[0].data  # Tenta acessar data
                    elif hasattr(selfie_upload.value[0], 'value'):
                        file_content = selfie_upload.value[0].value  # Tenta acessar value
                    else:
                        # Último recurso: tentativa de conversão direta
                        print("Tentando conversão direta do objeto para bytes...")
                        file_content = bytes(selfie_upload.value[0])
                else:
                    print("Formato de upload não reconhecido.")
                    return
                
                selfie_content = file_content
                print(f"Selfie carregada! Tamanho: {len(file_content) / 1024:.1f} KB")
            except Exception as e:
                print(f"Erro ao processar selfie: {e}")
                print(f"Tipo do objeto: {type(selfie_upload.value)}")
                if isinstance(selfie_upload.value, tuple) and len(selfie_upload.value) > 0:
                    print(f"Tipo do primeiro elemento: {type(selfie_upload.value[0])}")
                    print(f"Atributos disponíveis: {dir(selfie_upload.value[0])}")
                selfie_content = None

# Manipulador para o botão de verificação
### Adicione este código após a função on_selfie_upload_change

# Manipulador para o botão de verificação
def on_verify_button_click(b):
    with verify_output:
        clear_output()
        if not document_content:
            print("Erro: Nenhum documento válido foi carregado!")
            return
        
        if not selfie_content:
            print("Erro: Nenhuma selfie foi carregada!")
            return
        
        print("Verificando identidade...")
        print("-" * 50)
        
        # Verificação com threshold padrão
        match, similarity = compare_selfie_with_document(selfie_content, document_content)
        confidence = round((1 - similarity) * 100, 2)
        
        if match:
            print(f"✅ VERIFICAÇÃO CONCLUÍDA: As imagens correspondem à mesma pessoa!")
            print(f"Nível de confiança: {confidence}%")
            print("-" * 50)
            print("USUÁRIO VERIFICADO - Acesso liberado aos recursos exclusivos para fãs FURIA")
        else:
            print(f"❌ VERIFICAÇÃO FALHOU: As imagens não correspondem ou houve um erro.")
            print(f"Nível de confiança: {confidence}%")
            print("-" * 50)
            print("Recomendações:")
            print("- Tente fazer upload de uma selfie com melhor iluminação")
            print("- Certifique-se que seu rosto está claramente visível")
            print("- Verifique se o documento possui uma foto nítida")


# Registra os manipuladores de eventos
selfie_upload.observe(on_selfie_upload_change, names='value')
verify_button.on_click(on_verify_button_click)  # Adicione o evento de clique do botão

FileUpload(value=(), accept='.jpg,.jpeg,.png,.bmp', description='Upload de Selfie')

Output()

Button(description='Verificar Identidade', style=ButtonStyle())

Output()

### Seção 7: Relatório Final

In [12]:
generate_report_button = widgets.Button(description="Gerar Relatório do Fã")
report_output = widgets.Output()

display(generate_report_button, report_output)

def on_report_button_click(b):
    with report_output:
        clear_output()
        
        # Verifica se os dados essenciais foram preenchidos
        if not name_input.value or not cpf_input.value or not document_content:
            print("⚠️ Por favor, preencha os dados essenciais (nome, CPF válido e documento) antes de gerar o relatório.")
            return
            
        # Cabeçalho do relatório
        print("=" * 60)
        print("🔥 FURIA FAN INSIGHTS - RELATÓRIO DE FÃ 🔥")
        print("=" * 60)
        
        # Dados pessoais
        print(f"👤 Nome: {name_input.value}")
        print(f"🏠 Endereço: {address_input.value if address_input.value else 'Não informado'}")
        print(f"📄 CPF: {cpf_input.value} {'✅ Válido' if validate_cpf(cpf_input.value) else '❌ Inválido'}")
        
        # Verificação de documento
        print(f"🪪 Documento: {'✅ Verificado' if document_content else '❌ Não verificado'}")
        
        # Verificação de identidade
        identity_verified = False
        if document_content and selfie_content:
            match, similarity = compare_selfie_with_document(selfie_content, document_content)
            confidence = round((1 - similarity) * 100, 2)
            identity_verified = match
            print(f"🤳 Verificação de Identidade: {'✅ Verificada' if match else '❌ Falhou'} (Confiança: {confidence}%)")
        else:
            print("🤳 Verificação de Identidade: ❌ Não realizada")
        
        # Interesses
        if interests_input.value:
            interests = [i.strip() for i in interests_input.value.split(',')]
            print("\n📊 PERFIL DE INTERESSE")
            print("-" * 60)
            for interest in interests:
                print(f"• {interest}")
        
        # Perfil de engajamento
        print("\n🌟 NÍVEL DE ENGAJAMENTO")
        print("-" * 60)
        
        engagement_score = 0
        factors = []
        
        # Calcula pontuação com base nos dados fornecidos
        if identity_verified:
            engagement_score += 30
            factors.append("Identidade verificada (+30)")
            
        if validate_cpf(cpf_input.value):
            engagement_score += 10
            factors.append("CPF válido (+10)")
            
        if interests_input.value:
            engagement_score += 15
            factors.append("Interesses informados (+15)")
            
        if social_input.value:
            engagement_score += 20
            factors.append("Redes sociais informadas (+20)")
            
        if profile_link_input.value:
            if validate_profile_link(profile_link_input.value, interests_input.value):
                engagement_score += 25
                factors.append("Perfil externo verificado (+25)")
            else:
                engagement_score += 10
                factors.append("Perfil externo informado (+10)")
        
        # Exibe os fatores que afetaram a pontuação
        for factor in factors:
            print(f"• {factor}")
            
        # Categoria de fã
        print("\n🏆 CLASSIFICAÇÃO DO FÃ")
        print("-" * 60)
        
        if engagement_score >= 80:
            fan_category = "FÃ SUPERFURIA 🔥🔥🔥"
            benefits = [
                "Acesso prioritário a ingressos para eventos",
                "Descontos exclusivos de 20% em produtos oficiais",
                "Possibilidade de encontros com jogadores",
                "Acesso a áreas VIP em eventos"
            ]
        elif engagement_score >= 50:
            fan_category = "FÃ LEAL 🔥🔥"
            benefits = [
                "Descontos de 10% em produtos oficiais",
                "Pré-venda de ingressos para eventos",
                "Newsletter exclusiva com conteúdo dos bastidores"
            ]
        else:
            fan_category = "FÃ INICIANTE 🔥"
            benefits = [
                "Newsletter com novidades da FURIA",
                "Acesso a conteúdos exclusivos no site"
            ]
        
        print(f"Pontuação de Engajamento: {engagement_score}/100")
        print(f"Categoria: {fan_category}")
        
        print("\n💎 BENEFÍCIOS DISPONÍVEIS")
        for benefit in benefits:
            print(f"• {benefit}")
            
        print("\n" + "=" * 60)
        print("Relatório gerado por FURIA Fan Insights - " + str(datetime.datetime.now().strftime("%d/%m/%Y %H:%M")))
        print("=" * 60)

generate_report_button.on_click(on_report_button_click)

Button(description='Gerar Relatório do Fã', style=ButtonStyle())

Output()

### Seção 8: Salvar Dados

In [17]:
save_button = widgets.Button(description="Salvar Dados de Fã")
save_output = widgets.Output()

display(save_button, save_output)

def on_save_button_click(b):
    with save_output:
        clear_output()
        
        if not name_input.value or not cpf_input.value:
            print("⚠️ Nome e CPF são obrigatórios para salvar os dados.")
            return
            
        try:
            import json
            import os
            
            # Cria diretório de dados se não existir
            os.makedirs("fan_data", exist_ok=True)
            
            # Cria um dicionário com os dados do fã
            fan_data = {
                "name": name_input.value,
                "address": address_input.value,
                "cpf": cpf_input.value,
                "interests": interests_input.value,
                "social_media": social_input.value,
                "profile_link": profile_link_input.value,
                "document_provided": document_content is not None,
                "selfie_provided": selfie_content is not None,
                "timestamp": str(datetime.datetime.now())
            }
            
            # Salva os dados em um arquivo JSON
            filename = f"fan_data/fan_{cpf_input.value.replace('.', '').replace('-', '')}.json"
            with open(filename, "w") as f:
                json.dump(fan_data, f, indent=4)
                
            print(f"✅ Dados do fã {name_input.value} salvos com sucesso!")
            
        except Exception as e:
            print(f"❌ Erro ao salvar dados: {e}")
            
save_button.on_click(on_save_button_click)

Output()

Output()

## Instruções de Uso

1. **Dados Pessoais**: Insira seu nome, endereço e CPF nos campos fornecidos. O sistema validará automaticamente o CPF.

2. **Interesses**: Descreva seus interesses em esports, como jogos favoritos (CS2, League of Legends), times (ex: FURIA), eventos e compras relacionadas. Separe múltiplos interesses com vírgulas.

3. **Upload de Documento**: Faça upload de uma imagem de um documento de identificação com foto. A IA detectará se há um rosto na imagem.

4. **Redes Sociais**: Insira seus perfis de redes sociais (ex: @usuario_twitter, @usuario_instagram). Separe múltiplos perfis com vírgulas. O sistema simulará a leitura de interações relacionadas a esports.

5. **Links de Perfis**: Insira links para seus perfis em sites de esports (Steam, FACEIT, etc). Separe múltiplos links com vírgulas. A IA verificará se o conteúdo é relevante com base nos seus interesses.

6. **Verificação de Identidade**: Faça upload de uma selfie e clique em "Verificar Identidade" para comparar seu rosto com o documento fornecido. A verificação mostrará se as imagens correspondem à mesma pessoa.

7. **Relatório do Fã**: Após preencher os dados necessários, clique em "Gerar Relatório do Fã" para visualizar uma análise completa do seu perfil, incluindo nível de engajamento e benefícios disponíveis.

8. **Salvar Dados**: Para armazenar suas informações no sistema, clique em "Salvar Dados de Fã". Os dados serão salvos em formato JSON para uso futuro.